In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import feature_manager as fma
import classifier.cross_validator as mcv 


2023-02-03 17:20:07.778134: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-03 17:20:07.778233: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-03 17:20:07.778246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-02-03 17:20:09.736560: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1965 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 3GB, pci bus id: 0000:01:00.

In [2]:
symbol = "BTCUSDT"
trade_tf = "4h"
macro_tf = "1d"
super_tf = "1w"
granular_tf = "1m"

fm = fma.FeatureManager(target_col="trade_signal")

fm.import_trading_data(symbol=symbol,trade_timeframe=trade_tf)
fm.import_macro_data(symbol=symbol,macro_timeframe=macro_tf)
fm.import_super_data(symbol=symbol,super_timeframe=super_tf)
fm.import_granular_data(symbol=symbol,granular_timeframe=granular_tf)

Imported trading data from ../data/BTCUSDT-4h.csv with 11760 rows
Imported macro data from ../data/BTCUSDT-1d.csv with 1963 rows
Imported super macro data from ../data/BTCUSDT-1w.csv with 247 rows
Imported granular data from ../nocommit/BTCUSDT-1m.csv with 2817999 rows


In [3]:
tp = 0.042
sl = 0.042
md = 14

fm.prepare_trade_forward_data(
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
)

Scanning 14 future timeframes to build trade signal: 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 
Label producing completed. 
 Value counts:
0    3849
1    3973
2    3938
Name: trade_signal, dtype: int64


In [7]:
candlestick_features = [
    "CDL2CROWS",
    "CDL3BLACKCROWS",
    "CDL3INSIDE",
    "CDL3LINESTRIKE",
    "CDL3OUTSIDE",
    "CDL3STARSINSOUTH",
    "CDL3WHITESOLDIERS",
    "CDLABANDONEDBABY",
    "CDLADVANCEBLOCK",
    "CDLBELTHOLD",
    "CDLBREAKAWAY",
    "CDLCLOSINGMARUBOZU",
    "CDLCONCEALBABYSWALL",
    "CDLCOUNTERATTACK",
    "CDLDARKCLOUDCOVER",
    "CDLDOJI",
    "CDLDOJISTAR",
    "CDLDRAGONFLYDOJI",
    "CDLENGULFING",
    "CDLEVENINGDOJISTAR",
    "CDLEVENINGSTAR",
    "CDLGAPSIDESIDEWHITE",
    "CDLGRAVESTONEDOJI",
    "CDLHAMMER",
    "CDLHANGINGMAN",
    "CDLHARAMI",
    "CDLHARAMICROSS",
    "CDLHIGHWAVE",
    "CDLHIKKAKE",
    "CDLHIKKAKEMOD",
    "CDLHOMINGPIGEON",
    "CDLIDENTICAL3CROWS",
    "CDLINNECK",
    "CDLINVERTEDHAMMER",
    "CDLKICKING",
    "CDLKICKINGBYLENGTH",
    "CDLLADDERBOTTOM",
    "CDLLONGLEGGEDDOJI",
    "CDLLONGLINE",
    "CDLMARUBOZU",
    "CDLMATCHINGLOW",
    "CDLMATHOLD",
    "CDLMORNINGDOJISTAR",
    "CDLMORNINGSTAR",
    "CDLONNECK",
    "CDLPIERCING",
    "CDLRICKSHAWMAN",
    "CDLRISEFALL3METHODS",
    "CDLSEPARATINGLINES",
    "CDLSHOOTINGSTAR",
    "CDLSHORTLINE",
    "CDLSPINNINGTOP",
    "CDLSTALLEDPATTERN",
    "CDLSTICKSANDWICH",
    "CDLTAKURI",
    "CDLTASUKIGAP",
    "CDLTHRUSTING",
    "CDLTRISTAR",
    "CDLUNIQUE3RIVER",
    "CDLUPSIDEGAP2CROWS",
    "CDLXSIDEGAP3METHODS"
]

external_features = [
    "hashrate",
    "fed_rate",
    "gold",
    "nasdaq",
    "sp500",
    "google_trend",   
]

features = [
    # "returns",
    # "dir",
    "sma_3_10",
    "sma_7_30",
    "sma_14_50",
    "sma_28_90",         
    # "boll",     
    # "boll7",
    # "boll14",
    # "boll21",
    # "min",      
    # "min7",      
    # "min14",
    # "min21",
    # "max",      
    # "max7",      
    # "max14",
    # "max21",
    # "mom",
    # "mom7",      
    # "mom14",
    # "mom21",
    # "vol",      
    # "vol7",      
    # "vol14",
    # "vol21",
    "obv",      
    # "mfi7",     
    "mfi14",
    # "mfi21",
    "rsi7",      
    "rsi14",
    "rsi30",
    # "adx7",      
    # "adx14",
    # "adx21",
    # "roc",      
    # "roc7",      
    # "roc14",
    # "roc21",
    # "atr7",      
    # "atr14",
    # "atr21",
    # "bop",      
    # "ad",       
    # "adosc",     
    # "trange",    
    # "ado",       
    # "willr7",     
    # "willr14",
    # "willr21",
    # "dx7",     
    # "dx14",
    # "dx21",
    # "trix",     # 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    # "ultosc",   # Ultimate Oscillator
    # "high",
    # "low",
]

macro_features = [
    # "returns",
    # "dir",    
    "sma_3_10",
    "sma_7_30",
    "sma_14_50",
    "sma_28_90",           
    # "boll",     
    # "boll7",
    # "boll14",
    # "boll21",
    # "min",      
    # "min7",      
    # "min14",
    # "min21",
    # "max",      
    # "max7",      
    # "max14",
    # "max21",
    # "mom",
    # "mom7",      
    # "mom14",
    # "mom21",
    # "vol",      
    # "vol7",      
    # "vol14",
    # "vol21",
    "obv",      
    # "mfi7",     
    "mfi14",
    # "mfi21",
    "rsi7",      
    "rsi14",
    "rsi30",
    # "rsi60",
    # "rsi90",
    # "adx7",      
    # "adx14",
    # "adx21",
    # "roc",      
    # "roc7",      
    # "roc14",
    # "roc21",
    # "atr7",      
    # "atr14",
    # "atr21",
    # "bop",      
    # "ad",       
    # "adosc",     
    # "trange",    
    # "ado",       
    # "willr7",     
    # "willr14",
    # "willr21",
    # "dx7",     
    # "dx14",
    # "dx21",
    # "trix",     # 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    # "ultosc",   # Ultimate Oscillator
    # "high",
    # "low",
] 

super_features = [
    # "returns",
    # "dir",  
    "sma_3_10",
    "sma_7_30",
    "sma_14_50",
    # "sma_28_90",           
    # "boll",     
    # "boll7",
    # "boll14",
    # "boll21",
    # "min",      
    # "min7",      
    # "min14",
    # "min21",
    # "max",      
    # "max7",      
    # "max14",
    # "max21",
    # "mom",
    # "mom7",      
    # "mom14",
    # "mom21",
    # "vol",      
    # "vol7",      
    # "vol14",
    # "vol21",
    "obv",      
    # "mfi7",     
    "mfi14",
    # "mfi21",
    "rsi7",      
    "rsi14",
    "rsi30",
    # "rsi60",
    # "rsi90",
    # "adx7",      
    # "adx14",
    # "adx21",
    # "roc",      
    # "roc7",      
    # "roc14",
    # "roc21",
    # "atr7",      
    # "atr14",
    # "atr21",
    # "bop",      
    # "ad",       
    # "adosc",     
    # "trange",    
    # "ado",       
    # "willr7",     
    # "willr14",
    # "willr21",
    # "dx7",     
    # "dx14",
    # "dx21",
    # "trix",     # 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    # "ultosc",   # Ultimate Oscillator
    # "high",
    # "low",
] 

fm.build_features(
    lags=14,
    macro_lags=7,
    super_lags=3,
    features=features + candlestick_features + external_features,
    macro_features=macro_features + candlestick_features,
    super_features=super_features + candlestick_features,
)

Calculating external features ...
Calculating TA indicators of trade timeframe
Calculating TA indicators of macro timeframe
Calculating TA indicators of super timeframe
Calculating candlestick of trade timeframe
Calculating candlestick of macro timeframe
Calculating candlestick of super timeframe

Adding features for trade timeframe with lags 14: sma_3_10, sma_7_30, sma_14_50, sma_28_90, obv, mfi14, rsi7, rsi14, rsi30, CDL2CROWS, CDL3BLACKCROWS, CDL3INSIDE, CDL3LINESTRIKE, CDL3OUTSIDE, CDL3STARSINSOUTH, CDL3WHITESOLDIERS, CDLABANDONEDBABY, CDLADVANCEBLOCK, CDLBELTHOLD, CDLBREAKAWAY, CDLCLOSINGMARUBOZU, CDLCONCEALBABYSWALL, CDLCOUNTERATTACK, CDLDARKCLOUDCOVER, CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI, CDLENGULFING, CDLEVENINGDOJISTAR, CDLEVENINGSTAR, CDLGAPSIDESIDEWHITE, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN, CDLHARAMI, CDLHARAMICROSS, CDLHIGHWAVE, CDLHIKKAKE, CDLHIKKAKEMOD, CDLHOMINGPIGEON, CDLIDENTICAL3CROWS, CDLINNECK, CDLINVERTEDHAMMER, CDLKICKING, CDLKICKINGBYLENGTH, CDLLADDERB

In [ ]:
fm.plot_trade_signal()

In [12]:
import importlib
importlib.reload(mcv)

<module 'classifier.cross_validator' from '/home/andy/CryptoTradingPlatform/TraderRobot/project/classifier/cross_validator.py'>

In [13]:

cv = mcv.CrossValidator()
cv.cross_validate_multi_svm(
    fm=fm,
    write_to_file=True,
    kernel="rbf",
    C=10,
    gamma=0.001,
    fold_number=3,
    cache_size=3000,
    tol=1e-3,
)


CROSS VALIDATION PARAMS
classifier:svm
C:10
kernel:rbf
degree:4
gamma:0.001
randome_state:1
probability:True
fold_numeber:3
decision_function_shape:ovr
class_weight:balanced
split_type:time_series_split
rebalance:None



DATA:
Total rows: 7594
Label 0.0: 2659(35.01%)
Label 1.0: 2421(31.88%)
Label 2.0: 2514(33.11%)
Splitting the data...

>>>>>> FOLD 1


DATA IN FOLD
Train: 1898, Validation: 1898

Train:
Label 0.0: 698(36.78%)
Label 1.0: 573(30.19%)
Label 2.0: 627(33.03%)

Test:
Label 0.0: 740(38.99%)
Label 1.0: 682(35.93%)
Label 2.0: 476(25.08%)
[LibSVM].*.*
optimization finished, #iter = 2095
obj = -5164.280954, rho = -1.944865
nSV = 777, nBSV = 544
Total nSV = 777
.*.*
optimization finished, #iter = 2264
obj = -4975.661264, rho = -1.314836
nSV = 765, nBSV = 512
Total nSV = 765
.*.*
optimization finished, #iter = 2128
obj = -5194.295615, rho = -1.779245
nSV = 778, nBSV = 530
Total nSV = 778
.*.*
optimization finished, #iter = 2497
obj = -5160.557862, rho = -1.481681
nSV = 791, nBSV = 